In [1]:
import pandas as pd
import requests
import regex as re
import numpy as np
import datetime as dt
import networkx as nx
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import plotly.express as px
import yfinance as yf
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima.utils import ndiffs
import pmdarima as pm
import pandas_datareader as web
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from keras import regularizers
import itertools
import math
from pmdarima.arima import auto_arima


In [3]:
sp_assets=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0] 
assets=sp_assets['Symbol'].str.replace('.', '-').tolist()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
stock_df={}
for stock in assets:
    df=yf.download(stock,start='2024-01-01',end='2024-04-06')
    df_frq=df.asfreq(pd.infer_freq(df.index))['Close']
    df_frq=df_frq.fillna(method='ffill')
    X_train=[]
    X_test=[]
    crct_windows=0
    total_windows=0
    losses={}
    for i in range(30,len(df_frq)):
        total_windows+=1
        X_train=df_frq.iloc[i-30:i]
        X_test=df_frq[i:i+1]
        adf_pvalue=adfuller(X_train)[1]
        d=0
        if adf_pvalue>0.05:
            X_train_diff=X_train.copy()
            while True:
                X_train_diff=X_train_diff.diff().dropna()
                adf_pvalue=adfuller(X_train_diff)[1]
                d+=1
                if adf_pvalue<=0.05:
                    break         
        p=q=range(0,12)
        pq=list(itertools.product(p,q))
        aic=math.inf
        best_params=(0,d,0)
        for i in pq:
            try:
                arima = ARIMA(X_train,order=(i[0],d,i[1]))
                arima_fit = arima.fit()
                aic_=arima_fit.aic
                if aic_<aic:
                    aic=aic_
                    best_params=(i[0],d,i[1])
            except:
                continue
        arima = ARIMA(X_train,order=best_params)
        arima_fit = arima.fit()
        prediction_df=arima_fit.get_forecast(X_test.index[0]).summary_frame()
        if X_test.iloc[0]>=prediction_df.loc[X_test.index[0]]['mean_ci_lower'] and X_test.iloc[0]<=prediction_df.loc[X_test.index[0]]['mean_ci_upper']:
            crct_windows+=1
            losses[X_test.index[0].strftime('%Y-%m-%d')]=[X_test.iloc[0]-prediction_df.loc[X_test.index[0]]['mean'],X_test.iloc[0],prediction_df.loc[X_test.index[0]]['mean']]
    stock_df[stock]=[crct_windows,total_windows,losses]
    

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
